In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
libraries = ["dhl", "numpy", "ngsolve"]
df = pd.DataFrame()
for library in libraries:
    try:
        df = pd.concat([df, pd.read_csv(f"results_{library}.csv")])
    except Exception as e:
        print(e)
        
df.head()

In [ ]:
df_grouped = (
    df.groupby(["labels", "matrix_size"])["gmacs"]
    .agg(['mean', 'std', 'count'])
)
df_grouped['ci'] = 1.96 * df_grouped['std'] / np.sqrt(df_grouped['count'])
df_grouped['ci_lower'] = df_grouped['mean'] - df_grouped['ci']
df_grouped['ci_upper'] = df_grouped['mean'] + df_grouped['ci']

df_grouped = df_grouped.reset_index()
df_grouped.head()

## Matplotlib

In [ ]:
plt.figure(figsize=(10, 6))

for label in df['labels'].unique():
    thread_data = df[df['labels'] == label]
    plt.scatter(thread_data['matrix_size'], thread_data['gmacs'], label=label)
    
    thread_data_grouped = df_grouped[df_grouped["labels"] == label]
    # Add mean line and a single filled confidence interval for each thread
    plt.plot(thread_data_grouped["matrix_size"], thread_data_grouped["mean"], label=f'Mean {label}')
    plt.fill_between(thread_data_grouped["matrix_size"], thread_data_grouped["ci_lower"], thread_data_grouped["ci_upper"], alpha=0.2)

plt.xlabel('Matrix Size')
plt.ylabel('GMAC/s')
plt.title('Matrix Matrix Multiplication')
plt.legend()
plt.grid()
# plt.yscale('log') 
plt.show()

## Plotly

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig = go.Figure()

for label in df['labels'].unique():
    thread_data = df[df['labels'] == label]
    fig.add_trace(go.Scatter(x=thread_data['matrix_size'], y=thread_data['gmacs'], mode='markers', name=label))
    
    thread_data_grouped = df_grouped[df_grouped["labels"] == label]
    # Add mean line and a single filled confidence interval for each thread
    fig.add_trace(go.Scatter(x=thread_data_grouped["matrix_size"], y=thread_data_grouped["mean"], mode='lines', line=dict(dash='dash'), name=f'Mean {label}'))
    fig.add_trace(go.Scatter(x=thread_data_grouped["matrix_size"], y=thread_data_grouped["ci_upper"], mode='lines', line=dict(dash='dash'), showlegend=False, line_color = 'rgba(0,0,0,0)'))
    fig.add_trace(go.Scatter(x=thread_data_grouped["matrix_size"], y=thread_data_grouped["ci_lower"], mode='lines',fill='tonexty', fillcolor='rgba(68, 68, 68, 0.3)', line=dict(dash='dash'), name=f'95% CI {label}', line_color = 'rgba(0,0,0,0)'))
    
fig.update_layout(
    xaxis_title='Matrix Size', 
    yaxis_title='GMAC/s', 
    title='Matrix Matrix Multiplication',)

fig.show()

In [ ]:
fig = px.bar(df_grouped, x="matrix_size", y="mean", color="labels", barmode='group')
fig.update_layout(
    xaxis_title='Matrix Size', 
    yaxis_title='GMAC/s', 
    title='Matrix Matrix Multiplication',
)